In [1]:
libOfInterest = 'safe_app'

In [2]:
libOfInterest = 'safe_authenticator'

In [3]:
with open('../safenet/extracted_headers/' + libOfInterest + '_function_declarations') as f:
    functionDeclarations=f.read()

In [4]:
oneline=functionDeclarations.splitlines()[3]

In [5]:
oneline

'void create_acc(char* account_locator, char* account_password, char* invitation, void* user_data, void (*o_disconnect_notifier_cb)(void* user_data), void (*o_cb)(void* user_data, FfiResult* result, Authenticator* authenticator));'

In [6]:
import re

In [7]:
startOfFunction=re.compile('.*?\(')

parameters=re.compile('\(.*\)')

functions=re.compile(', void \(\*.*?\)\(.*?\)')

In [8]:

nameOfFunction=startOfFunction.findall(oneline)[0][5:-1]
allParameters=parameters.findall(oneline)[0][1:-1]
callbackfunctions=functions.findall(oneline)
CallParameters=allParameters

In [9]:
nameOfFunction

'create_acc'

In [10]:
CallParameters

'char* account_locator, char* account_password, char* invitation, void* user_data, void (*o_disconnect_notifier_cb)(void* user_data), void (*o_cb)(void* user_data, FfiResult* result, Authenticator* authenticator)'

In [11]:

for oneCallback in callbackfunctions:
    print(oneCallback)
    CallParameters=CallParameters.replace(oneCallback,'')

, void (*o_disconnect_notifier_cb)(void* user_data)
, void (*o_cb)(void* user_data, FfiResult* result, Authenticator* authenticator)


In [12]:
CallParameters

'char* account_locator, char* account_password, char* invitation, void* user_data'

In [13]:
def getParameterDict(callParameters):
    mydict={}
    for onePar in callParameters:
        datatype,name = onePar.split()
        mydict[name]=datatype
    return mydict

In [14]:
callbackDict={}
for oneCallback in callbackfunctions:
    callbackDict[re.findall('\(\*.*?\)',oneCallback)[0][2:-1]]=getParameterDict(re.findall('\(.*?\)',oneCallback.replace(' const',''))[-1][1:-1].split(', '))

In [15]:
oneline

'void create_acc(char* account_locator, char* account_password, char* invitation, void* user_data, void (*o_disconnect_notifier_cb)(void* user_data), void (*o_cb)(void* user_data, FfiResult* result, Authenticator* authenticator));'

In [16]:
callbackfunctions

[', void (*o_disconnect_notifier_cb)(void* user_data)',
 ', void (*o_cb)(void* user_data, FfiResult* result, Authenticator* authenticator)']

In [17]:
callbackDict

{'o_disconnect_notifier_cb': {'user_data': 'void*'},
 'o_cb': {'user_data': 'void*',
  'result': 'FfiResult*',
  'authenticator': 'Authenticator*'}}

In [18]:
[(lambda x: x.split()[-1])(x) for x in CallParameters.split(',')]

['account_locator', 'account_password', 'invitation', 'user_data']

In [19]:
def getFunctionDefPy(name,parameters,praefix,callbackNames):
    line=praefix + 'def ' + 'self._'+name + '('
    innerStuff=''
    for x in [(lambda x: x.split()[-1])(x) for x in parameters.split(',')]:
         innerStuff += x + ', '
    innerStuff = innerStuff[:-2]
    line = line + innerStuff 
    for one in callbackNames:
        line += ', ' + one+'=None'
    line = line +', safenetLib=self.lib.' + libOfInterest + '):'
    return line,innerStuff

In [20]:
def getFunctionCommentPy(name,parameters,praefix):
    line=praefix + 'def '+name + '('
    for x in [(lambda x: x.split()[0])(x) for x in parameters.split(',')]:
        line += x + ', '
    line = line[:-2]+', safenetLib=self.lib.' + libOfInterest + '):'
    return line

In [21]:
CallParameters

'char* account_locator, char* account_password, char* invitation, void* user_data'

In [23]:
callbackDict

{'o_disconnect_notifier_cb': {'user_data': 'void*'},
 'o_cb': {'user_data': 'void*',
  'result': 'FfiResult*',
  'authenticator': 'Authenticator*'}}

In [24]:
CallParameters

'char* account_locator, char* account_password, char* invitation, void* user_data'

In [25]:
callbackfunctions[0][7:]

'(*o_disconnect_notifier_cb)(void* user_data)'

In [26]:
callbackfunctions

[', void (*o_disconnect_notifier_cb)(void* user_data)',
 ', void (*o_cb)(void* user_data, FfiResult* result, Authenticator* authenticator)']

In [27]:
def printMethodDefinition(callParameters_string,callbackfunctions,callbackNames):
    
    types = [(lambda x: x.split()[0])(x) for x in callParameters_string.split(',')]
    names = [(lambda x: x.split()[-1])(x) for x in callParameters_string.split(',')]
    callParameters={}
    for i,currType in enumerate(types):
        callParameters[names[i]]=currType
    
    #now the doc string
    tempstring='    """ '
    for onepar in callParameters:
        if callParameters[onepar] == 'void*':
            tempstring += '[any]' + ', '
        elif callParameters[onepar] == 'char*':
            tempstring += 'bytes' + ', '
        else:
            tempstring += callParameters[onepar] + ', '
    for onepar in callbackfunctions:
        tempstring += '[function], '
    
    print(tempstring[:-2])
    
    tempstring='        '
    for onepar in callParameters:
        tempstring += callParameters[onepar] + ' ' + onepar + ', '
    
    print(tempstring[:-2])
    
    print()
    
    print('        > callback functions:')
    
    for oneCallback in callbackfunctions:
        tempstring='        '  + str(oneCallback[7:])
        print(tempstring)
    print('    """')
    

In [28]:
CallParameters

'char* account_locator, char* account_password, char* invitation, void* user_data'

In [29]:
nameOfFunction

'create_acc'

In [30]:
[(lambda x: '_' + nameOfFunction + '_' + x)(x) for x in callbackDict.keys()]

['_create_acc_o_disconnect_notifier_cb', '_create_acc_o_cb']

In [31]:
callbackDict.keys()

dict_keys(['o_disconnect_notifier_cb', 'o_cb'])

In [32]:
callbackDict.keys()

dict_keys(['o_disconnect_notifier_cb', 'o_cb'])

In [33]:

praefix = '    '
print(praefix + '@safeUtils.safeThread')
callbackNames = [(lambda x: '_' + nameOfFunction + '_' + x)(x) for x in callbackDict.keys()]
funDef, innerParameters = getFunctionDefPy(nameOfFunction,CallParameters,praefix,callbackDict.keys())
print(funDef)
print('')
printMethodDefinition(CallParameters,callbackfunctions,nameOfFunction)

praefix = praefix + '    '

libCall = 'safenetLib.'+nameOfFunction+'('+innerParameters
for callback in callbackDict:
    declaration = praefix + f'@ffi.callback("void('
    for innerparam in callbackDict[callback]:
        declaration += callbackDict[callback][innerparam] + ' ,'
    declaration = declaration[:-2] + ')")'
    print(declaration)
    
    
    definition = praefix + 'def _' + nameOfFunction + '_' + str(callback) +'('
    for innerparam in callbackDict[callback]:
        definition += innerparam + ' ,'
    definition = definition[:-2] + '):'
    print(definition)
    print(praefix + '    ' + 'if ' + callback + ':')
    print(praefix + '    ' + '    ' + callback + '()')
    
    print()
    print()
    
    libCall += ', _' + nameOfFunction + '_' + str(callback)
    
print(praefix+libCall + ')')

    @safeUtils.safeThread
    def self._create_acc(account_locator, account_password, invitation, user_data, o_disconnect_notifier_cb=None, o_cb=None, safenetLib=self.lib.safe_authenticator):

    """ bytes, bytes, bytes, [any], [function], [function]
        char* account_locator, char* account_password, char* invitation, void* user_data

        > callback functions:
        (*o_disconnect_notifier_cb)(void* user_data)
        (*o_cb)(void* user_data, FfiResult* result, Authenticator* authenticator)
    """
        @ffi.callback("void(void*)")
        def _create_acc_o_disconnect_notifier_cb(user_data):
            if o_disconnect_notifier_cb:
                o_disconnect_notifier_cb()


        @ffi.callback("void(void* ,FfiResult* ,Authenticator*)")
        def _create_acc_o_cb(user_data ,result ,authenticator):
            if o_cb:
                o_cb()


        safenetLib.create_acc(account_locator, account_password, invitation, user_data, _create_acc_o_disconnect_notifier_cb, 

In [121]:
innerData=None
@ffi.callback("void(void* , FfiResult* , App* app)")
def result_app_registered(user_data, result, app):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    returnDict = ffi.from_handle(user_data)
    returnDict['app']=app

NameError: name 'ffi' is not defined

In [107]:
type([])

list

In [236]:
from threading import Thread

def threader(function):
    def innerThread(*args,**kwargs):
        oneThread=Thread(target=function,args=args,kwargs=kwargs)
        oneThread.start()
    return innerThread

In [11]:
import time

In [232]:
@threader
def print3():
    for i in range(3):
        time.sleep(1)
        print(i)
        

In [234]:
ls

ffiCallGenerator.ipynb  safeUtils.py


In [6]:

from threading import Thread
# making threads for SAFE
def thread(function):
    def innerThread(*args,**kwargs):
        oneThread=Thread(target=function,args=args,kwargs=kwargs)
        oneThread.start()
    return innerThread

In [1]:
import safeUtils

In [2]:
safeUtils

<module 'safeUtils' from '/home/riddim/py/pySafe/littleHelpers/safeUtils.py'>

In [9]:
import time

In [10]:
@safeUtils.safeThread
def print3():
    for i in range(3):
        time.sleep(1)
        print(i)
        

In [11]:
print3()
print3()
print3()

0
0
0
1
1
1
2
2
2


In [199]:
myLock = Lock()

1
2


In [89]:
testVar = 3

In [90]:
testVar

3

In [172]:
myLock.release()

RuntimeError: release unlocked lock

In [101]:
myLock.locked()

False

In [94]:
testVar = testVar+3

In [95]:
testVar

6